<a href="https://colab.research.google.com/github/nunocesarsa/SENSECO_School_2021/blob/main/ColabNotebooks/SENSECO_03_PROSAIL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Preparing google colab

- This notebook just details how to load PROSAIL into google colab and how to set up a simulation of Sentinel 2 data



In [1]:
#mounting google drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Installing packages

In [ ]:
#Installing PROSAIL
!pip install prosail

#latin hypercube stuff
!pip install lhsmdu

#this package as a number of functions to deal with hyperspectral data
!pip install pysptools

#Importing packages

In [3]:
#General purpose: 
import matplotlib.pyplot as plt
import numpy
import numpy as np
from mpl_toolkits.mplot3d import Axes3D
from scipy import stats

#the beutiful R like data frame
import pandas as pd

#PROSPECT+SAIL Radiative transfer mode package
import prosail

#Sampling design package
import lhsmdu

#a few more stuff for random
import random as rdm
import math

#package to for operations on spectral data
import pysptools as sptool 
from pysptools import distance

## Custom functions

- These functions have multiple purposes from simplifying the interaction with PROSAIL to converting the input data into S2 datasets.

- They can appear a bit complex because they are lenghty but they are simpler than they might seem at first sight

### Custom PROSAIL call

- Expects as input the biophysical parameters we want to model, returns the hyperspectral response. 

- Only function is to just simplify the usage of prosail.run_prosail

In [ ]:
def custom_prosail(cab,lai,sol_zen,inc_zen,raa):
  
  import prosail
  #"default" parameters
  cm = 0.005
  cw = 0.005
  n= 1.
  car=10.
  cbrown=0.01
  typelidf=1 #this is the default option
  lidfa = -1 #leaf angle distribution parameter a and b
  lidfb=-0.15
  hspot= 0.01 #hotspot parameters 

  #sun and viewing angle
  #tts=30. #observation and solar position parameters
  #tto=10. 
  #psi=0.

  #for now i put them by hand but they should be an input of a custom function
  tts=sol_zen #solar zenith angle
  tto=inc_zen #sensor zenith angle
  psi=raa

  

  rho_out = prosail.run_prosail(n,
                                 cab,
                                 car,
                                 cbrown,
                                 cw,
                                 cm,
                                 lai,
                                 lidfa,
                                 hspot,
                                 tts,tto,psi,
                                 typelidf, lidfb,
                                 prospect_version="D",
                                 factor='SDR', 
                                 rsoil=.5, psoil=.5)
  return(rho_out)

test_rho =  custom_prosail(20,3,30,30,30)
print(test_rho.shape)

#Optional plotting
plt.plot(test_rho)

### Hyperspectral to Sentinel 2

This function convert an array output from prosail.run_prosail to Sentinel 2 A or B data (depending on the sensor). 

S2 Response functions: https://sentinels.copernicus.eu/web/sentinel/user-guides/sentinel-2-msi/document-library/-/asset_publisher/Wk0TKajiISaR/content/sentinel-2a-spectral-responses

Note: for this exercise we will assume everything is Sentinel 2A


In [ ]:
#here you should change so it points to your file
filepath="/content/drive/MyDrive/SENSECO_S2Data/S2_Responses_S2A.csv"

def Prosail2S2(path2csv,spectra_input):

  #pointing needed packages
  import pandas as pd
  import numpy as np
  #upload a S2_Response.csv from https://earth.esa.int/web/sentinel/user-guides/sentinel-2-msi/document-library/-/asset_publisher/Wk0TKajiISaR/content/sentinel-2a-spectral-responses

  s2_table = pd.read_csv(path2csv,sep=";",decimal=",") #check if this is proper, regarding the sep and dec
  #chekc which row you are actually extracting

  s2_table_sel = s2_table[s2_table['SR_WL'].between(400,2500)] #selects all values between 400 and 2500
  spectra_input_df = pd.DataFrame(data=spectra_input,columns=["rho"],index=s2_table_sel.index) #transforms the input array into a pandas df with the column name rho and row.index = to the original input table

  rho_s2 = s2_table_sel.multiply(spectra_input_df['rho'],axis="index") #calculates the numerator
  w_band_sum = s2_table_sel.sum(axis=0,skipna = True) #calculates the denominator

  output = (rho_s2.sum(axis=0)/w_band_sum).rename_axis("ID").values #runs the weighted mean and converts the output to a numpy array

  return output[1:] #removes the first value because it represents the wavelength column

test_rho_s2 = Prosail2S2(filepath,test_rho)
print(test_rho_s2.shape)

#13 Band order:
#B1, 2 , 3 (..) 8, 8A, 9, 10, 11, 12

#Bands we want: 2 ,3 ,4 , 5, 6, 7, 8A, 11, 12

#Selecting only the bands of interest is then trivial:
test_rho_s2_sel = test_rho_s2[[1,2,3,4,5,6,7,11,12],]


#Optional plotting
plt.figure(0)
plt.plot(test_rho_s2,"o" )
plt.figure(1)
plt.plot(test_rho_s2_sel,"o")

### Generating a table of spectra data

Takes as input a pandas dataframe with the input data and uses that to call the prosail and stack everything neatly into an array which can then be used elsewhere

In [ ]:

def Gen_spectra_data(input_param_table,doPlot=False,bandlist=[0,1,2,3,4,5,6,7,8,9,10,11,12]):
  k = 1
  #pd_train_traits=traits
  #print(range(len(traits)))
  for i in range(len(input_param_table)):

    #edit this section accordingly
    filepath = "/content/drive/MyDrive/SENSECO_S2Data/S2_Responses_S2A.csv"

    #vegetation parametrs
    cab_t = input_param_table["cab"][i]
    lai_t = input_param_table["lai"][i]

    #observation parameters
    sol_zen_t = input_param_table["sol_zen"][i]
    inc_zen_t = input_param_table["inc_zen"][i]
    raa_t     = input_param_table["raa"][i]

    if k == 1:

      tr_rho_s = custom_prosail(cab_t,lai_t,sol_zen_t,inc_zen_t,raa_t)
      tr_rho_s = Prosail2S2(filepath,tr_rho_s)[bandlist,]

      if doPlot == True:
        x = bandlist
        plt.plot ( x, tr_rho_s)
        #plt.legend(loc='best')
      
    if k > 1:
      tr_rho_t = custom_prosail(cab_t,lai_t,sol_zen_t,inc_zen_t,raa_t)
      tr_rho_t = Prosail2S2(filepath,tr_rho_t)[bandlist,]
      tr_rho_s = np.vstack((tr_rho_s,tr_rho_t))

      if doPlot == True:
        plt.plot ( x, tr_rho_t)

    k = k+1

  rho_samples=tr_rho_s
  return rho_samples


#generating a trait table is a bit tricky 

#First we generate a random matrix of values between 0 and 1
#the input of the function is (N,M), N - number of columns, M - number of samples
LHS_train = lhsmdu.createRandomStandardUniformMatrix(5,100)

#then we convert this into a Pandas table and rename it
pd_param_input = pd.DataFrame.transpose(pd.DataFrame(LHS_train))
pd_param_input.columns = ["cab","lai","sol_zen","inc_zen","raa"]

#now we can just plut it into a line betwene maximum and minimum 
#notice: dont use 0 or unresonably wrong input paramaters because that might cause NaN responses

#pd_trait["cab"]=pd_trait["cab"]*max_cab+min_cab
#vegetation parameters
pd_param_input["cab"]    =pd_param_input["cab"]*80+20
pd_param_input["lai"]    =pd_param_input["lai"]*6+1

#observation parameters - for now very small and basic - it varies between 29 and 31
pd_param_input["sol_zen"]=pd_param_input["sol_zen"]*2+29
pd_param_input["inc_zen"]=pd_param_input["inc_zen"]*2+29
pd_param_input["raa"]    =pd_param_input["raa"]*2+29

#chcke the shape
print(pd_param_input.shape)

#test the function
test_spectra = Gen_spectra_data(pd_param_input,doPlot=True)


In [ ]:
test_spectra = Gen_spectra_data(pd_param_input,doPlot=True,bandlist=[1,2,3,4,5,6,7,11,12])

#Additive noise

This noise function simulates noise in function of the signal magnitude

Adapted from equation 4: https://www.mdpi.com/2072-4292/7/8/10321/html#B31-remotesensing-07-10321



In [8]:
def combined_noise(ref,sigma):

  #ref is an input vector and sigma is the standar deviation of the noise - there is a possibilityu of values going over over 1 or lower than 0 But those values are corrected in the end. 
  
  mp_noise = ref*(1 + stats.truncnorm.rvs(0-sigma*4,0+sigma*4,loc=0,scale=sigma*2,size=ref.shape[0]))
  ad_noise = 0+stats.truncnorm.rvs(0-sigma*2,0+sigma*2,loc=0,scale=sigma,size=ref.shape[0])
  out_ref = mp_noise+ ad_noise

  #making everything that goes under, be 0 or 1
  out_ref[out_ref>1]=1
  out_ref[out_ref<0]=0

  return out_ref